In [2]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

# path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
# path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Unclassified Splits/Unattributable Part "
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [4]:
excel_path_unattributable = path_equity_precision_llm_folder + "//llm training/epldat03_Unattributable Data.csv"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")
execfile(path_equity_precision_llm_repo + "/functions/standardize_population.py")
execfile(path_equity_precision_llm_repo + "/functions/adjusted_source_population_match.py")

In [5]:
input_unattributable = pd.read_csv(excel_path_unattributable)
input_unattributable.head()

,PMID,MeSH,Abstract,Title,year
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010


In [17]:
import pandas as pd
import re

n_json_splits = 11
# Concatenate multiple CSVs
results = None
for scenario in range(1, n_json_splits + 1):
    file_path = f"{path_equity_precision_llm_folder}/llm training/Unclassified Splits/Unattributable Part {scenario}_results.csv"
    temp_df = pd.read_csv(file_path)
    temp_df = temp_df[['pmid', 'title', 'gpt_precision_medicine', 'gpt_diabetes', 'gpt_primary_study', 'gpt_source_population']]

    if results is None:
        results = temp_df
    else:
        results = pd.concat([results, temp_df], ignore_index=True)

# Merge datasets
merged_df_unattributable = input_unattributable.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df_unattributable['gpt_source_population'] = merged_df_unattributable['gpt_source_population'].apply(standardize_population)

In [18]:
# Check the output below
merged_df_unattributable.head()

,PMID,MeSH,Abstract,Title,year,pmid,title,gpt_precision_medicine,gpt_diabetes,gpt_primary_study,gpt_source_population
0,18008027,"Albuminuria, Angiotensin II Type 1 Receptor Bl...",The purpose of the present study was to invest...,Renoprotective effects of telmisartan on renal...,2013,18008027.0,Renoprotective effects of telmisartan on rena...,no,yes,yes,Unknown
1,18205795,"Antigens, Tumor-Associated, Carbohydrate, Bloo...",CA 72-4 is one of the blood group carbohydrate...,CA 72-4 levels in patients with type 2 diabete...,2011,18205795.0,CA 72-4 levels in patients with type 2 diabet...,no,yes,yes,Unknown
2,18378359,"Age Distribution, Age Factors, Aged, 80 and ov...",The aim of this study was to evaluate the asso...,Vascular risk factors and dementia in the gene...,2010,18378359.0,Vascular risk factors and dementia in the gen...,no,yes,yes,we
3,18626579,"Bioreactors, Biotechnology, Fermentation, Gluc...",The production of propionic acid by Propioniba...,Kinetic analysis and pH-shift control strategy...,2010,18626579.0,Kinetic analysis and pH-shift control strateg...,no,no,yes,Unknown
4,18651247,"Bioreactors, Butyric Acid, Cells, Immobilized,...",Butyric acid has many applications in chemical...,Production of butyric acid from glucose and xy...,2010,18651247.0,Production of butyric acid from glucose and x...,no,no,yes,Unknown


In [ ]:
merged_df_unattributable.to_csv(path_equity_precision_llm_folder + '/llm training/epl05_output_Unattributable.csv', index=False)